# To Pointcloud converter

In [1]:
import numpy as np
import imageio
import struct
import os
import glob
import yaml

from dictionary_manager import get_index
from dictionary_manager import get_serial_dictionary

In [2]:
def get_pointcloud(color_image,depth_image,camera_intrinsics):
    """ creates 3D point cloud of rgb images by taking depth information
        input : color image: numpy array[h,w,c], dtype= uint8
                depth image: numpy array[h,w] values of all channels will be same
        output : camera_points, color_points - both of shape(no. of pixels, 3)
    """

    image_height = depth_image.shape[0]
    image_width = depth_image.shape[1]
    pixel_x,pixel_y = np.meshgrid(np.linspace(0,image_width-1,image_width),
                                  np.linspace(0,image_height-1,image_height))
    camera_points_x = np.multiply(pixel_x-camera_intrinsics[0,2],depth_image/camera_intrinsics[0,0])
    camera_points_y = np.multiply(pixel_y-camera_intrinsics[1,2],depth_image/camera_intrinsics[1,1])
    camera_points_z = depth_image
    camera_points = np.array([camera_points_x,camera_points_y,camera_points_z]).transpose(1,2,0).reshape(-1,3)

    color_points = color_image.reshape(-1,3)

    # Remove invalid 3D points (where depth == 0 and color == 0)
    valid_depth_color_ind = np.where((depth_image.flatten() > 0) & (color_image[:,:,0].flatten() > 0))[0] # Check pixels greaters than 0 in both color and depth images
    camera_points = camera_points[valid_depth_color_ind,:]
    color_points = color_points[valid_depth_color_ind,:]

    return camera_points,color_points

In [3]:
def write_pointcloud(filename,xyz_points,rgb_points=None):

    """ creates a .pkl file of the point clouds generated
    """

    assert xyz_points.shape[1] == 3,'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8)*255
    assert xyz_points.shape == rgb_points.shape,'Input RGB colors should be Nx3 float array and have same size as input XYZ points'

    # Write header of .ply file
    fid = open(filename,'wb')
    fid.write(bytes('ply\n', 'utf-8'))
    fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
    fid.write(bytes('element vertex %d\n'%xyz_points.shape[0], 'utf-8'))
    fid.write(bytes('property float x\n', 'utf-8'))
    fid.write(bytes('property float y\n', 'utf-8'))
    fid.write(bytes('property float z\n', 'utf-8'))
    fid.write(bytes('property uchar red\n', 'utf-8'))
    fid.write(bytes('property uchar green\n', 'utf-8'))
    fid.write(bytes('property uchar blue\n', 'utf-8'))
    fid.write(bytes('end_header\n', 'utf-8'))

    # Write 3D points to .ply file
    for i in range(xyz_points.shape[0]):
        fid.write(bytearray(struct.pack("fffccc",xyz_points[i,0],xyz_points[i,1],xyz_points[i,2],
                                        rgb_points[i,0].tobytes(),rgb_points[i,1].tobytes(),
                                        rgb_points[i,2].tobytes())))
    fid.close()

In [4]:
def convert_to_pointcloud(path2, patient):
    images = glob.glob(path2 + '/*.png')
    
    # Get the indices from the images folder corresponding to color and depth images
    color_index = get_index('segment', images)
    depth_index = get_index('depth', images)
    
    # Obtain the path of the corresponding indices for color and depth images
    color_files = [images[i] for i in color_index]
    depth_files = [images[i] for i in depth_index]
    
    file_name = path2 + '/' + 'intrinsics_matrix.yaml'
    with open(file_name, 'r') as file:
        intrinsics_matrix = yaml.safe_load(file)
        
    file_name = path2 + '/' + 'depth_scale.yaml'
    with open(file_name, 'r') as file:
        depth_scale = yaml.safe_load(file)
        
    serial_color_path = get_serial_dictionary(color_files, intrinsics_matrix)
    serial_depth_path = get_serial_dictionary(depth_files, intrinsics_matrix) 
    
    for serial in intrinsics_matrix:
        color_data = imageio.imread(serial_color_path[serial])
        depth_data = imageio.imread(serial_depth_path[serial])
        depth_data = depth_data * depth_scale[serial]
        camera_intrinsics = np.asarray(intrinsics_matrix[serial])

        output_filename = path2 + '/' + patient + '_' + str(serial) + '.ply'
        print("Creating the point Cloud file at : ", output_filename )
        camera_points, color_points = get_pointcloud(color_data, depth_data, camera_intrinsics)
        write_pointcloud(output_filename, camera_points, color_points)

## Reference

* [Realsense - Create pointCloud](https://gist.github.com/Shreeyak/9a4948891541cb32b501d058db227fff)